# Pump it up
[Pump it Up 網站](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)

之所以會用ipython notebook是因爲要測試非常多不同模型，一直preprocessing很浪費時間。並且模型訊息不夠明確也是很麻煩的事情。

並且想利用notebook把重要的訊息就存放在這裏面，比較方便思考。

recorded year - construction year

grid search

In [2]:
import csv
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import xgboost as xgb
from sklearn import model_selection
from datetime import datetime
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from argparse import ArgumentParser

VALI = 0.9
X_PATH = '../../ML2017_data/bump/x.csv'
Y_PATH = '../../ML2017_data/bump/y.csv'
X_TEST = '../../ML2017_data/bump/test_x.csv'

## 利用panda來做資料處理

In [84]:
df_x = pd.read_csv(X_PATH,parse_dates=True)
df_x_test = pd.read_csv(X_TEST,parse_dates=True)
df_y = pd.read_csv(Y_PATH)

df=df_x.append(df_x_test)

df.construction_year=pd.to_numeric(df.construction_year)
df.loc[df.construction_year <= 0, df.columns=='construction_year'] = 1950


df['operation_time']=df.date_recorded.apply(pd.to_datetime)-df.construction_year.apply(lambda x: pd.to_datetime(x,format='%Y'))# df['timeago']=df.timeago.apply(lambda x: int(x)/100000000000.)
df['operation_time']=df.operation_time.apply(lambda x: x.total_seconds()/ (365.25*24*60*60*1e9))
df.loc[df['operation_time'] < 0,df.columns=='operation_time'] = 63.92060232717317
# df['date_recorded'] = df.date_recorded.apply(lambda x: (pd.to_datetime(x) - datetime(1976,1,1)).total_seconds) 

df['month']=pd.to_datetime(df.date_recorded).dt.month
df['public_meeting'] = df.public_meeting.apply(lambda x: str(x))
df['permit'] = df.public_meeting.apply(lambda x: str(x))

factorschange=[x for x in df.columns if x not in ['id','latitude','longitude','gps_height','date_recorded','construction_year','month','operation_time']]

for factor in factorschange:
    values_factor=df_x[factor].value_counts()
    lessthen=values_factor[values_factor < 20]
    listnow=df_x.installer.isin(list(lessthen.keys()))
    df_x.loc[listnow,factor] = 'Others'
    # names to numbers
    print(factor)
    df[factor].fillna('NaN', inplace = True )
    df[factor] = preprocessing.LabelEncoder().fit_transform(df[factor])

df.population = df.population.apply(lambda x: np.log10(x+1))

    
a= df[df["longitude"] < 1]
a.iloc[:,df.columns == "latitude"]= np.nan
a.iloc[:,df.columns == "longitude"]= np.nan
df[df["longitude"] < 1] = a
df["longitude"] = df.groupby("region_code").transform(lambda x: x.fillna(x.mean())).longitude
df["latitude"] = df.groupby("region_code").transform(lambda x: x.fillna(x.mean())).latitude

a= df[df["gps_height"] < 1]
a.iloc[:,df.columns == "gps_height"]= np.nan
df[df["gps_height"] < 1] = a
df["gps_height"] = df.groupby("region_code").transform(lambda x: x.fillna(x.mean())).gps_height

df=df.fillna(df.mean())

factors=[u'funder', u'gps_height',
       u'installer', u'longitude', u'latitude', u'wpt_name',
       u'basin', u'subvillage', u'region', u'region_code', u'district_code',
       u'lga', u'ward', u'population', u'public_meeting',
       u'scheme_management', u'scheme_name', u'permit', u'construction_year',
       u'extraction_type', u'extraction_type_group', u'extraction_type_class',
       u'management', u'management_group', u'payment', u'payment_type',
       u'water_quality', u'quality_group', u'quantity', u'quantity_group',
       u'source', u'source_type', u'source_class', u'waterpoint_type',
       u'waterpoint_type_group','month','operation_time']

df = df[factors]
df_x_test = df[len(df_x):]
df_x = df[:len(df_x)]
df_y = df_y.status_group
pickle.dump(df_x, open('x', 'wb'))
pickle.dump(df_y, open('y', 'wb'))
pickle.dump(df_x_test, open('x_test', 'wb'))

print('finish preprocessing')

amount_tsh
funder
installer
wpt_name
num_private
basin
subvillage
region
region_code
district_code
lga
ward
population
public_meeting
recorded_by
scheme_management
scheme_name
permit
extraction_type
extraction_type_group
extraction_type_class
management
management_group
payment
payment_type
water_quality
quality_group
quantity
quantity_group
source
source_type
source_class
waterpoint_type
waterpoint_type_group


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


finish preprocessing


In [96]:
x = np.array(pickle.load(open('x', 'rb')))
y = np.array(pickle.load(open('y', 'rb')))
x_test = np.array(pickle.load(open('x_test', 'rb')))

print(x[0])
print(x_test[0])
print(y[0])

[  1.54900000e+03   1.39000000e+03   1.70700000e+03   3.49380928e+01
  -9.85632177e+00   4.56830000e+04   1.00000000e+00   1.31160000e+04
   3.00000000e+00   1.00000000e+01   5.00000000e+00   5.10000000e+01
   1.42900000e+03   2.04139269e+00   1.00000000e+00   8.00000000e+00
   2.38900000e+03   1.00000000e+00   1.99900000e+03   3.00000000e+00
   1.00000000e+00   0.00000000e+00   7.00000000e+00   4.00000000e+00
   2.00000000e+00   0.00000000e+00   6.00000000e+00   2.00000000e+00
   1.00000000e+00   1.00000000e+00   8.00000000e+00   6.00000000e+00
   0.00000000e+00   1.00000000e+00   1.00000000e+00   3.00000000e+00
   1.21971253e-08]
[  3.45000000e+02   1.99600000e+03   4.26000000e+02   3.52907992e+01
  -4.05969643e+00   2.66900000e+03   0.00000000e+00   9.93400000e+03
   8.00000000e+00   2.00000000e+01   3.00000000e+00   6.30000000e+01
   1.80000000e+01   2.50514998e+00   1.00000000e+00   4.00000000e+00
   2.00800000e+03   1.00000000e+00   2.01200000e+03   9.00000000e+00
   6.00000000e+

## RandomForeset + Self-train

In [85]:
np.random.seed(3)

x = np.array(pickle.load(open('x', 'rb')))
y = np.array(pickle.load(open('y', 'rb')))
x_test = np.array(pickle.load(open('x_test', 'rb')))

scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
x_test = scaler.transform(x_test)
pickle.dump(scaler, open('scaler', 'wb'))
print('Finish normizatilon')

TRAIN = int(VALI * len(x))
x_t = x[:TRAIN]
y_t = y[:TRAIN]
x_v = x[TRAIN:]
y_v = y[TRAIN:]

#pca = PCA(n_components = 36)%%!
#pca.fit(all_x)
#x = pca.transform(x)
#x_test = pca.transform(x_test)
#print('Finish PCA')

# 一次self-train 拿出 0.65以上的
for i in range(2):
    print(len(x_test))
    clf = RandomForestClassifier(n_estimators = 1500,
            bootstrap = True,
            n_jobs = -1, 
            min_samples_split=7,
            oob_score = True)
    clf.fit(x, y)
    score = clf.oob_score_
    print(score)
    test_ans = clf.predict_proba(x_test)
    ans = clf.predict(x_test)
#     pickle.dump(clf, open('./model/{:.4f}'.format(score), 'wb'))
    train_l = np.where(test_ans > 0.65)
    #print(test_ans.shape)
    #print(x_test.shape)
    #print('ans shape', ans.shape)
    x_test = np.array(x_test)
    self_train_x = x_test[train_l[0]]
    self_train_y = ans[train_l[0]]
    x_test = np.delete(x_test, train_l[0], axis = 0)
    x = np.concatenate((x, self_train_x))
    y = np.concatenate((y, self_train_y))
print('Finish training')

label_list = ['functional', 'functional needs repair', 'non functional']
x_test = np.array(pickle.load(open('x_test', 'rb')))
x_test = scaler.transform(x_test)
ans = clf.predict(x_test)
x_test = np.array(list(csv.reader(open(X_TEST, 'r'))))
id_list = x_test[1:, :1].reshape(x_test.shape[0]-1)
print(id_list)
with open('ans.csv', 'w') as f:
    writer = csv.writer(f, lineterminator = '\n')
    writer.writerow(['id', 'status_group'])
    for i in range(len(ans)):
        writer.writerow([id_list[i], ans[i]])

Finish normizatilon
14850
0.819393939394
3704
0.849233124486
Finish training
['50785' '51630' '17168' ..., '28749' '33492' '68707']


seed = 10

|min_samples_split|accuracy|
|:-:|:-:|
|7|0.819074074074|
|8|0.819259259259|
|9|0.819057239057|

seed = 3

|min_samples_split|accuracy|
|:-:|:-:|
|7|0.819393939394|
|8|0.819141414141|
|9|0.819057239057|

In [72]:
x = pickle.load(open('x', 'rb'))
y = pickle.load(open('y', 'rb'))
x_test = pickle.load(open('x_test', 'rb'))

scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
x_test = scaler.transform(x_test)
pickle.dump(scaler, open('scaler', 'wb'))
print('Finish normizatilon')

TRAIN = int(VALI * len(x))
x_t = x[:TRAIN]
y_t = y[:TRAIN]
x_v = x[TRAIN:]
y_v = y[TRAIN:]

#pca = PCA(n_components = 36)%%!
#pca.fit(all_x)
#x = pca.transform(x)
#x_test = pca.transform(x_test)
#print('Finish PCA')

# 一次self-train 拿出 0.65以上的
for i in range(2):
    print(len(x_test))
    clf = RandomForestClassifier(n_estimators = 1500,
            n_jobs = -1, 
            min_samples_split=10,
            oob_score = True)
    clf.fit(x, y)
    score = clf.oob_score_
    print(score)
    test_ans = clf.predict_proba(x_test)
    ans = clf.predict(x_test)
#     pickle.dump(clf, open('./model/{:.4f}'.format(score), 'wb'))
    train_l = np.where(test_ans > 0.65)
    #print(test_ans.shape)
    #print(x_test.shape)
    #print('ans shape', ans.shape)
    x_test = np.array(x_test)
    self_train_x = x_test[train_l[0]]
    self_train_y = ans[train_l[0]]
    x_test = np.delete(x_test, train_l[0], axis = 0)
    x = np.concatenate((x, self_train_x))
    y = np.concatenate((y, self_train_y))
print('Finish training')

label_list = ['functional', 'functional needs repair', 'non functional']
x_test = np.array(pickle.load(open('x_test', 'rb')))
x_test = scaler.transform(x_test)
ans = clf.predict(x_test)
x_test = np.array(list(csv.reader(open(X_TEST, 'r'))))
id_list = x_test[1:, :1].reshape(x_test.shape[0]-1)
print(id_list)
with open('ans.csv', 'w') as f:
    writer = csv.writer(f, lineterminator = '\n')
    writer.writerow(['id', 'status_group'])
    for i in range(len(ans)):
        writer.writerow([id_list[i], label_list[ans[i]]])

ValueError: could not convert string to float: '2011-03-23'

xgb我最後的希望。

|做法|error|
|:-:|:-:|
|全feature| 0.186886 |
|全feature + scaler| 0.188182 |
|去除wpt_name和num_private| 0.187121 |
|去除wpt_name和num_private + scaler| 0.187239   |
|去除所有上面feature| 0.187576  |
|去除所有上面feature + scaler| 0.188838  |
|['id' 'wpt_name' 'num_private' 'recorded_by']| 0.186465 |
|['id' 'wpt_name' 'num_private' 'recorded_by' 'payment_type' 'water_quality' 'scheme_management' 'quantity_group' 'waterpoint_type_group']| 0.186162 |

In [28]:
x = np.array(pickle.load(open('x', 'rb')))
y = np.array(pickle.load(open('y', 'rb')))
randomize = np.arange(len(x))
np.random.shuffle(randomize)
x = x[randomize]
y = y[randomize]
x_test = pickle.load(open('x_test', 'rb'))
all_x = np.concatenate((x, x_test))

# scaler = StandardScaler()
# scaler.fit(all_x)
# x = scaler.transform(x)
# x_test = scaler.transform(x_test)
# pickle.dump(scaler, open('scaler', 'wb'))
# print('Finish normizatilon')

TRAIN = int(VALI * len(x))
x_t = x[:TRAIN]
y_t = y[:TRAIN]
x_v = x[TRAIN:]
y_v = y[TRAIN:]

all_xtrain = xgb.DMatrix(x, y)
dtrain = xgb.DMatrix(x_t, y_t)
dval = xgb.DMatrix(x_v, y_v)
dtest = xgb.DMatrix(x_test)

xgb_par = {
        'objective' : 'multi:softmax',
        'booster' : 'gbtree',
        'num_class' : 3,
        'eval_matric' : "merror",
        'max_depth' : 14,
        'colsample_bytree' : .4,
        }
cv_out = xgb.cv(
        xgb_par,
        all_xtrain,
        num_boost_round = 1000,
        nfold = 5,
        early_stopping_rounds = 20,
        verbose_eval = 50,
        show_stdv = False)
print(cv_out)
print('Round:', len(cv_out))
rounds = len(cv_out)

# clf = xgb.train(xgb_par, 
#         dtrain = dtrain,
#         evals = [(dval, 'eval')],
#         num_boost_round = 1000,
#         early_stopping_rounds = 20,
#         )

print(clf.best_ntree_limit)
ans = clf.predict(dtest)
X_TEST = '/home/ymy1248/Code/ML2017_data/bump/test_x.csv'
x_test = np.array(list(csv.reader(open(X_TEST, 'r'))))
id_list = x_test[1:, :1].reshape(x_test.shape[0]-1)
print(id_list)
label_list = ['functional', 'functional needs repair', 'non functional']
with open('ans.csv', 'w') as f:
    writer = csv.writer(f, lineterminator = '\n')
    writer.writerow(['id', 'status_group'])
    for i in range(len(ans)):
        writer.writerow([id_list[i], label_list[int(ans[i])]])

[0]	train-merror:0.152626	test-merror:0.2233
[50]	train-merror:0.0094446	test-merror:0.189007
[100]	train-merror:0.0006942	test-merror:0.189242
    test-merror-mean  test-merror-std  train-merror-mean  train-merror-std
0           0.223300         0.005669           0.152626          0.002175
1           0.208872         0.004149           0.133733          0.006545
2           0.203973         0.002805           0.125059          0.003566
3           0.200792         0.002060           0.116936          0.001335
4           0.197071         0.001781           0.107807          0.003094
5           0.195757         0.002066           0.100572          0.004281
6           0.193603         0.002225           0.093127          0.002701
7           0.192795         0.002417           0.085787          0.002651
8           0.191700         0.003155           0.079979          0.002313
9           0.191684         0.002833           0.074928          0.002276
10          0.191801         0.

AttributeError: 'RandomForestClassifier' object has no attribute 'best_ntree_limit'

## Bagging for xgb

In [106]:
MODEL_NUM = 101

x = np.array(pickle.load(open('x', 'rb')))
y = np.array(pickle.load(open('y', 'rb')))
randomize = np.arange(len(x))
x_test = pickle.load(open('x_test', 'rb'))
all_x = np.concatenate((x, x_test))

scaler = StandardScaler()
scaler.fit(all_x)
x = scaler.transform(x)
x_test = scaler.transform(x_test)
pickle.dump(scaler, open('scaler', 'wb'))
print('Finish normizatilon')

label_encorder = LabelEncoder()
y = label_encorder.fit_transform(y)
pickle.dump(label_encorder, open('label_encoder', 'wb'))
print('Finish label')

VALI = 0.66

clf = RandomForestClassifier(n_estimators = 1500,
            bootstrap = True,
            n_jobs = -1, 
            min_samples_split=7,
            oob_score = True)
clf.fit(x, y)
score = clf.oob_score_
test_ans = clf.predict_proba(x_test)
ans = clf.predict(x_test)
#     pickle.dump(clf, open('./model/{:.4f}'.format(score), 'wb'))
train_l = np.where(test_ans > 0.65)
#print(test_ans.shape)
#print(x_test.shape)
#print('ans shape', ans.shape)
x_test = np.array(x_test)
self_train_x = x_test[train_l[0]]
self_train_y = ans[train_l[0]]
x_test = np.delete(x_test, train_l[0], axis = 0)
x = np.concatenate((x, self_train_x))
y = np.concatenate((y, self_train_y))

print('Finish self-train')

for model_i in range(MODEL_NUM):
    np.random.shuffle(randomize)
    x = x[randomize]
    y = y[randomize]
    TRAIN = int(VALI * len(x))
    x_t = x[:TRAIN]
    y_t = y[:TRAIN]
    x_v = x[TRAIN:]
    y_v = y[TRAIN:]


    dtrain = xgb.DMatrix(x_t, y_t)
    dval = xgb.DMatrix(x_v, y_v)

    xgb_par = {
            'objective' : 'multi:softprob',
            'booster' : 'gbtree',
            'eta' : 0.03,
            'subsample' : 0.8,
            'num_class' : 3,
            'max_depth' : 19,
            'colsample_bytree' : .5,
            }

    clf = xgb.train(xgb_par, 
        dtrain = dtrain,
        evals = [(dval, 'eval')],
        num_boost_round = 1000,
        early_stopping_rounds = 35,
        )
    
    pickle.dump(clf, open('./bagging/' + str(model_i), 'wb'))

Finish normizatilon
Finish label
Finish self-train
[0]	eval-merror:0.222569
Will train until eval-merror hasn't improved in 35 rounds.
[1]	eval-merror:0.206229
[2]	eval-merror:0.202169
[3]	eval-merror:0.199544
[4]	eval-merror:0.200584
[5]	eval-merror:0.197712
[6]	eval-merror:0.195979
[7]	eval-merror:0.195583
[8]	eval-merror:0.195484
[9]	eval-merror:0.195732
[10]	eval-merror:0.1939
[11]	eval-merror:0.194543
[12]	eval-merror:0.193454
[13]	eval-merror:0.192464
[14]	eval-merror:0.193108
[15]	eval-merror:0.192959
[16]	eval-merror:0.193553
[17]	eval-merror:0.19281
[18]	eval-merror:0.192315
[19]	eval-merror:0.191226
[20]	eval-merror:0.191276
[21]	eval-merror:0.190236
[22]	eval-merror:0.190186
[23]	eval-merror:0.189097
[24]	eval-merror:0.1888
[25]	eval-merror:0.189443
[26]	eval-merror:0.189493
[27]	eval-merror:0.189344
[28]	eval-merror:0.188849
[29]	eval-merror:0.189344
[30]	eval-merror:0.189146
[31]	eval-merror:0.188948
[32]	eval-merror:0.188998
[33]	eval-merror:0.188948
[34]	eval-merror:0.18

[20]	eval-merror:0.192464
[21]	eval-merror:0.192563
[22]	eval-merror:0.191672
[23]	eval-merror:0.19182
[24]	eval-merror:0.191919
[25]	eval-merror:0.190879
[26]	eval-merror:0.191028
[27]	eval-merror:0.191276
[28]	eval-merror:0.192018
[29]	eval-merror:0.191771
[30]	eval-merror:0.191969
[31]	eval-merror:0.191771
[32]	eval-merror:0.191573
[33]	eval-merror:0.190929
[34]	eval-merror:0.191127
[35]	eval-merror:0.19078
[36]	eval-merror:0.191325
[37]	eval-merror:0.191375
[38]	eval-merror:0.191523
[39]	eval-merror:0.191176
[40]	eval-merror:0.191077
[41]	eval-merror:0.190978
[42]	eval-merror:0.190681
[43]	eval-merror:0.190434
[44]	eval-merror:0.189443
[45]	eval-merror:0.189493
[46]	eval-merror:0.18979
[47]	eval-merror:0.18984
[48]	eval-merror:0.18984
[49]	eval-merror:0.189443
[50]	eval-merror:0.189097
[51]	eval-merror:0.189592
[52]	eval-merror:0.189394
[53]	eval-merror:0.189939
[54]	eval-merror:0.189741
[55]	eval-merror:0.189592
[56]	eval-merror:0.189394
[57]	eval-merror:0.189196
[58]	eval-merror:

[59]	eval-merror:0.187859
[60]	eval-merror:0.18771
[61]	eval-merror:0.187611
[62]	eval-merror:0.187314
[63]	eval-merror:0.187958
[64]	eval-merror:0.18776
[65]	eval-merror:0.187661
[66]	eval-merror:0.187265
[67]	eval-merror:0.187116
[68]	eval-merror:0.186869
[69]	eval-merror:0.187364
[70]	eval-merror:0.187661
[71]	eval-merror:0.187809
[72]	eval-merror:0.18771
[73]	eval-merror:0.187215
[74]	eval-merror:0.187314
[75]	eval-merror:0.187017
[76]	eval-merror:0.18672
[77]	eval-merror:0.186869
[78]	eval-merror:0.186671
[79]	eval-merror:0.186374
[80]	eval-merror:0.186869
[81]	eval-merror:0.186918
[82]	eval-merror:0.187413
[83]	eval-merror:0.187364
[84]	eval-merror:0.187463
[85]	eval-merror:0.187364
[86]	eval-merror:0.187661
[87]	eval-merror:0.18771
[88]	eval-merror:0.187364
[89]	eval-merror:0.187067
[90]	eval-merror:0.187215
[91]	eval-merror:0.187067
[92]	eval-merror:0.187215
[93]	eval-merror:0.187265
[94]	eval-merror:0.187116
[95]	eval-merror:0.187116
[96]	eval-merror:0.187463
[97]	eval-merror:

[103]	eval-merror:0.185878
[104]	eval-merror:0.185284
[105]	eval-merror:0.185136
[106]	eval-merror:0.185334
[107]	eval-merror:0.185037
[108]	eval-merror:0.184938
[109]	eval-merror:0.185136
[110]	eval-merror:0.185433
[111]	eval-merror:0.185136
[112]	eval-merror:0.185383
[113]	eval-merror:0.185086
[114]	eval-merror:0.184938
[115]	eval-merror:0.185631
[116]	eval-merror:0.185334
[117]	eval-merror:0.185532
[118]	eval-merror:0.185532
[119]	eval-merror:0.185433
[120]	eval-merror:0.185284
[121]	eval-merror:0.185383
[122]	eval-merror:0.185284
[123]	eval-merror:0.185433
[124]	eval-merror:0.185136
[125]	eval-merror:0.184789
[126]	eval-merror:0.18474
[127]	eval-merror:0.18474
[128]	eval-merror:0.184938
[129]	eval-merror:0.184839
[130]	eval-merror:0.184591
[131]	eval-merror:0.184641
[132]	eval-merror:0.184145
[133]	eval-merror:0.184641
[134]	eval-merror:0.185086
[135]	eval-merror:0.184789
[136]	eval-merror:0.184987
[137]	eval-merror:0.185136
[138]	eval-merror:0.18469
[139]	eval-merror:0.184987
[140

[108]	eval-merror:0.18979
[109]	eval-merror:0.189939
[110]	eval-merror:0.189642
[111]	eval-merror:0.189691
[112]	eval-merror:0.190137
[113]	eval-merror:0.190137
[114]	eval-merror:0.189889
[115]	eval-merror:0.189939
[116]	eval-merror:0.189889
[117]	eval-merror:0.190236
[118]	eval-merror:0.190137
[119]	eval-merror:0.190335
[120]	eval-merror:0.190285
[121]	eval-merror:0.190038
[122]	eval-merror:0.190285
[123]	eval-merror:0.190137
[124]	eval-merror:0.189939
[125]	eval-merror:0.190087
[126]	eval-merror:0.189889
[127]	eval-merror:0.189642
[128]	eval-merror:0.189642
[129]	eval-merror:0.190087
[130]	eval-merror:0.190038
[131]	eval-merror:0.190285
[132]	eval-merror:0.189889
[133]	eval-merror:0.190087
[134]	eval-merror:0.189889
[135]	eval-merror:0.189889
[136]	eval-merror:0.18984
[137]	eval-merror:0.18979
[138]	eval-merror:0.189988
[139]	eval-merror:0.190038
Stopping. Best iteration:
[104]	eval-merror:0.189295

[0]	eval-merror:0.226282
Will train until eval-merror hasn't improved in 35 rounds.
[

...

[7]	eval-merror:0.19697
[8]	eval-merror:0.196475
[9]	eval-merror:0.195534
[10]	eval-merror:0.195088
[11]	eval-merror:0.193801
[12]	eval-merror:0.193355
[13]	eval-merror:0.192959
[14]	eval-merror:0.191176
[15]	eval-merror:0.19187
[16]	eval-merror:0.192068
[17]	eval-merror:0.19182
[18]	eval-merror:0.191672
[19]	eval-merror:0.191424
[20]	eval-merror:0.191325
[21]	eval-merror:0.190137
[22]	eval-merror:0.190038
[23]	eval-merror:0.190186
[24]	eval-merror:0.189344
[25]	eval-merror:0.1888
[26]	eval-merror:0.188552
[27]	eval-merror:0.188998
[28]	eval-merror:0.1888
[29]	eval-merror:0.189245
[30]	eval-merror:0.189493
[31]	eval-merror:0.189542
[32]	eval-merror:0.188206
[33]	eval-merror:0.187562
[34]	eval-merror:0.187958
[35]	eval-merror:0.187859
[36]	eval-merror:0.187958
[37]	eval-merror:0.1888
[38]	eval-merror:0.187859
[39]	eval-merror:0.187958
[40]	eval-merror:0.187958
[41]	eval-merror:0.188354
[42]	eval-merror:0.18771
[43]	eval-merror:0.187958
[44]	eval-merror:0.187413
[45]	eval-merror:0.187314

[98]	eval-merror:0.191176
[99]	eval-merror:0.191028
[100]	eval-merror:0.191523
[101]	eval-merror:0.191424
[102]	eval-merror:0.191622
[103]	eval-merror:0.191375
[104]	eval-merror:0.191919
[105]	eval-merror:0.191672
[106]	eval-merror:0.191276
[107]	eval-merror:0.191176
[108]	eval-merror:0.191375
[109]	eval-merror:0.191077
[110]	eval-merror:0.191375
[111]	eval-merror:0.191077
[112]	eval-merror:0.191672
[113]	eval-merror:0.191424
[114]	eval-merror:0.191474
[115]	eval-merror:0.191028
[116]	eval-merror:0.191622
[117]	eval-merror:0.191969
[118]	eval-merror:0.191919
[119]	eval-merror:0.191721
[120]	eval-merror:0.19182
[121]	eval-merror:0.191622
[122]	eval-merror:0.191523
[123]	eval-merror:0.191474
[124]	eval-merror:0.191919
Stopping. Best iteration:
[89]	eval-merror:0.190632

[0]	eval-merror:0.226134
Will train until eval-merror hasn't improved in 35 rounds.
[1]	eval-merror:0.212022
[2]	eval-merror:0.204347
[3]	eval-merror:0.203357
[4]	eval-merror:0.202466
[5]	eval-merror:0.199594
[6]	eval-mer

[12]	eval-merror:0.196772
[13]	eval-merror:0.196376
[14]	eval-merror:0.196326
[15]	eval-merror:0.195039
[16]	eval-merror:0.195583
[17]	eval-merror:0.194593
[18]	eval-merror:0.193603
[19]	eval-merror:0.192959
[20]	eval-merror:0.192662
[21]	eval-merror:0.192414
[22]	eval-merror:0.192068
[23]	eval-merror:0.191919
[24]	eval-merror:0.191919
[25]	eval-merror:0.191672
[26]	eval-merror:0.191375
[27]	eval-merror:0.190879
[28]	eval-merror:0.190236
[29]	eval-merror:0.191523
[30]	eval-merror:0.190632
[31]	eval-merror:0.190483
[32]	eval-merror:0.190384
[33]	eval-merror:0.190434
[34]	eval-merror:0.190632
[35]	eval-merror:0.190731
[36]	eval-merror:0.190632
[37]	eval-merror:0.191077
[38]	eval-merror:0.191424
[39]	eval-merror:0.190681
[40]	eval-merror:0.18984
[41]	eval-merror:0.190236
[42]	eval-merror:0.18979
[43]	eval-merror:0.190236
[44]	eval-merror:0.18984
[45]	eval-merror:0.18979
[46]	eval-merror:0.189443
[47]	eval-merror:0.188651
[48]	eval-merror:0.189642
[49]	eval-merror:0.189493
[50]	eval-merror

[106]	eval-merror:0.189196
[107]	eval-merror:0.189493
[108]	eval-merror:0.189146
[109]	eval-merror:0.189097
[110]	eval-merror:0.189691
[111]	eval-merror:0.18979
[112]	eval-merror:0.189592
[113]	eval-merror:0.189245
[114]	eval-merror:0.189295
[115]	eval-merror:0.189097
[116]	eval-merror:0.188998
[117]	eval-merror:0.189097
[118]	eval-merror:0.188948
[119]	eval-merror:0.189542
[120]	eval-merror:0.189146
Stopping. Best iteration:
[85]	eval-merror:0.188701

[0]	eval-merror:0.230788
Will train until eval-merror hasn't improved in 35 rounds.
[1]	eval-merror:0.209497
[2]	eval-merror:0.205783
[3]	eval-merror:0.204149
[4]	eval-merror:0.202218
[5]	eval-merror:0.201872
[6]	eval-merror:0.198356
[7]	eval-merror:0.198406
[8]	eval-merror:0.197564
[9]	eval-merror:0.197019
[10]	eval-merror:0.19801
[11]	eval-merror:0.197217
[12]	eval-merror:0.196227
[13]	eval-merror:0.195088
[14]	eval-merror:0.194197
[15]	eval-merror:0.194444
[16]	eval-merror:0.194048
[17]	eval-merror:0.194444
[18]	eval-merror:0.194593
[

[153]	eval-merror:0.193504
[154]	eval-merror:0.193702
[155]	eval-merror:0.193504
[156]	eval-merror:0.193157
[157]	eval-merror:0.192959
[158]	eval-merror:0.193058
[159]	eval-merror:0.192959
[160]	eval-merror:0.193454
Stopping. Best iteration:
[125]	eval-merror:0.192711

[0]	eval-merror:0.223113
Will train until eval-merror hasn't improved in 35 rounds.
[1]	eval-merror:0.210685
[2]	eval-merror:0.202664
[3]	eval-merror:0.200485
[4]	eval-merror:0.199148
[5]	eval-merror:0.197415
[6]	eval-merror:0.196276
[7]	eval-merror:0.196227
[8]	eval-merror:0.194494
[9]	eval-merror:0.193306
[10]	eval-merror:0.194048
[11]	eval-merror:0.194197
[12]	eval-merror:0.194147
[13]	eval-merror:0.193999
[14]	eval-merror:0.193058
[15]	eval-merror:0.192266
[16]	eval-merror:0.192266
[17]	eval-merror:0.19182
[18]	eval-merror:0.191176
[19]	eval-merror:0.190582
[20]	eval-merror:0.190533
[21]	eval-merror:0.190137
[22]	eval-merror:0.189592
[23]	eval-merror:0.190038
[24]	eval-merror:0.189493
[25]	eval-merror:0.189493
[26]	e

## xgb bagging predict

In [107]:
x_test = pickle.load(open('x_test', 'rb'))
scaler = pickle.load(open('scaler', 'rb'))
label_encorder = pickle.load(open('label_encorder', 'rb'))
x_test = scaler.transform(x_test)
dtest = xgb.DMatrix(x_test)
ans = np.zeros((14850,3))
for i in range(MODEL_NUM):
    model = pickle.load(open('bagging/' + str(i), 'rb'))
    ans += model.predict(dtest, ntree_limit = model.best_ntree_limit)
ans = ans/MODEL_NUM
ans = np.argmax(ans, axis = 1)
ans = label_encorder.inverse_transform(ans)

X_TEST = '/home/ymy1248/Code/ML2017_data/bump/test_x.csv'
x_test = np.array(list(csv.reader(open(X_TEST, 'r'))))
id_list = x_test[1:, :1].reshape(x_test.shape[0]-1)
print(id_list)
with open('ans.csv', 'w') as f:
    writer = csv.writer(f, lineterminator = '\n')
    writer.writerow(['id', 'status_group'])
    for i in range(len(ans)):
        writer.writerow([id_list[i], ans[i]])

['50785' '51630' '17168' ..., '28749' '33492' '68707']
